# Sea Surface Temperature

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis, SST_remove_forced_signal
from maps import map_robinson, map_eq_earth, rect_polygon
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect, file_ex_ocn_lpd
from regions import boolean_mask, global_ocean, gl_ocean_rect, SST_index_bounds, boolean_mask
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA, dll_dims_names
from xr_regression import xr_linear_trends_2D, xr_linear_trend, ocn_field_regression, xr_lintrend, xr_quadtrend

# global yearly avg. SST map and trends

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc', decode_times=False)
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' , decode_times=False)
SST_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc' , decode_times=False)
SST_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpi.nc' , decode_times=False)
SST_yrly_had  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_had.nc' , decode_times=False)

### mean CTRL SST

In [ ]:
MASK  = boolean_mask(domain='ocn', mask_nr=0, rounded=True)
xa    = SST_yrly_ctrl.mean(dim='time').where(MASK)
fn    = f'{path_results}/SST/SST_ctrl_mean'
cm    = discrete_cmap(17, cmocean.cm.thermal)
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cm, minv=-2, maxv=32, label='SST [$^\circ$C]', filename=fn)

In [ ]:
%%time
# 2 min
SST_trend_ctrl, SST_trend_interc_ctrl = ocn_field_regression(SST_yrly_ctrl, run='ctrl')
SST_trend_rcp , SST_trend_interc_rcp  = ocn_field_regression(SST_yrly_rcp , run='rcp' )
SST_trend_lpd , SST_trend_interc_lpd  = ocn_field_regression(SST_yrly_lpd , run='lpd' )
SST_trend_lpi , SST_trend_interc_lpi  = ocn_field_regression(SST_yrly_lpi , run='lpi' )

In [ ]:
SST_yrly_lpd[:,100,100].plot()
(SST_trend_lpd[100,100]*SST_yrly_lpd.time + SST_trend_interc_lpd[100,100]).plot()

In [ ]:
cmap  = discrete_cmap(12, shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk'))

xa    = SST_trend_ctrl*100
fn    = f'{path_results}/SST/SST_trend_ctrl'
label = '100-299 SST trend [K/century]'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

xa    = SST_trend_rcp*100
fn    = f'{path_results}/SST/SST_trend_rcp'
label = '2000-2099 SST trend [K/century]'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

### GMST

In [ ]:
ds = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_ctrl.nc')

In [ ]:
ds[:,2000,1000].plot()

In [ ]:
SST_new[:,2000,1000].plot()

In [ ]:
# %%time
# # GMST: 5 sec for lpd/lpi, 1:37 rcp, <1 sec for had, 5 min for all
# for run in ['lpd', 'lpi', 'had', 'ctrl', 'rcp']:
#     SST_new, beta = SST_remove_forced_signal(run=run)
# for dts in ['AMO', 'SOM', 'TPI1', 'TPI2', 'TPI3']:
#     SST_new, beta = SST_remove_forced_signal(run='had', tres='yrly', detrend_signal=dts)

In [ ]:
beta_ctrl = xr.open_dataset(f'{path_samoc}/SST/SST_beta_GMST_yrly_ctrl.nc', decode_times=False)
beta_rcp  = xr.open_dataset(f'{path_samoc}/SST/SST_beta_GMST_yrly_rcp.nc' , decode_times=False)
beta_lpi  = xr.open_dataset(f'{path_samoc}/SST/SST_beta_GMST_yrly_lpi.nc' , decode_times=False)
beta_lpd  = xr.open_dataset(f'{path_samoc}/SST/SST_beta_GMST_yrly_lpd.nc' , decode_times=False)
beta_had  = xr.open_dataset(f'{path_samoc}/SST/SST_beta_GMST_yrly_had.nc' , decode_times=False)

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_ctrl.nc', decode_times=False)
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_rcp.nc' , decode_times=False)
SST_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc' , decode_times=False)
SST_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpi.nc' , decode_times=False)
SST_yrly_had  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_had.nc' , decode_times=False)

In [ ]:
SST_dt_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_ctrl.nc', decode_times=False)
SST_dt_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_rcp.nc' , decode_times=False)
SST_dt_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_lpd.nc' , decode_times=False)
SST_dt_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_lpi.nc' , decode_times=False)
SST_dt_yrly_had  = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_had.nc' , decode_times=False)

In [ ]:
betas = [beta_ctrl, beta_rcp, beta_lpi, beta_lpd, beta_had]
for i in range(5):
    f, ax = plt.subplots(1, 2, figsize=(8,3))
    betas[i].slope.plot(vmax=2, ax=ax[0])
    betas[i].forcing.plot(ax=ax[1])
    plt.tight_layout()

In [ ]:
beta_ctrl

In [ ]:
beta_ctrl.slope

In [ ]:
SST_dt_yrly_ctrl

In [ ]:
lat, lon = 20, 100
print(beta.slope[lat,lon].values)
beta.forcing.plot()
(SST_new[:,lat,lon]-SST_new[:,lat,lon].mean()).plot()
SST_new[:,lat,lon].plot(ls='--')
(beta.slope[lat,lon]*beta.forcing).plot(lw=5)


In [ ]:
xr_lintrend(SST_dt_yrly_lpd[:,lat,lon]).plot(ls='-')

In [ ]:
plt.scatter(beta_rcp.GMST, SST_ts)

In [ ]:
betas = [beta_ctrl, beta_rcp, beta_lpi, beta_lpd, beta_had]
domains = ['ocn', 'ocn', 'ocn_low', 'ocn_low', 'ocn_had']
for i in range(5):
    (depth, lat, lon) = dll_dims_names(domain=domains[i])
    print(depth, lat, lon)
    
    f, ax = plt.subplots(1, 2, figsize=(8,3))
#     betas[i].slope.plot(vmax=2, ax=ax[0])
    if i<4: betas[i].GMST.plot(ax=ax[0])
    if i==4: betas[i].tas.plot(ax=ax[0])
    plt.tight_layout()

### hiatus trend

In [ ]:
SST_trend_rcp_hiatus  = ocn_field_regression(SST_yrly_rcp[20:29,:,:])

In [ ]:
fn = f'{path_results}/SST/SST_trend_rcp_hiatus'
cmap = shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk')
label = 'SST trend [K/century]'
xa = SST_trend_rcp_hiatus*100
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-4, maxv=8, label=label, filename=fn)

# Ocean basins and SST index regions

In [ ]:
for file in [file_ex_ocn_ctrl, file_ex_ocn_lpd]:
    REGION_MASK = xr.open_dataset(file, decode_times=False).REGION_MASK
    fig = plt.figure(figsize=(8,5))
    cmap = discrete_cmap(12, 'hsv')
    ax  = fig.add_subplot(1, 1, 1, projection=ccrs.EqualEarth(central_longitude=200))
    ax.set_position([.02,.05,.96,.93])
    cax, kw = mpl.colorbar.make_axes(ax, location='bottom', pad=0.03, shrink=0.8)

    im = ax.pcolormesh(REGION_MASK.TLONG,
                       REGION_MASK.TLAT,
                       REGION_MASK.where(REGION_MASK>0),
                       cmap=cmap,
                       vmin=.5, vmax=12.5,
                       transform=ccrs.PlateCarree() )

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
    gl.ylocator = mpl.ticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xlocator = mpl.ticker.FixedLocator([-180, -120, -60, 0, 60, 120, 180])
    
    for mask in ['TPI1', 'TPI2', 'TPI3', 'SOM', 'AMO']:
        ax.add_patch(mpatches.Polygon(xy=rect_polygon(SST_index_bounds(mask)),
                                      facecolor='none',
                                      edgecolor='k',
                                      linewidth=2,
                                      zorder=2,
                                      transform=ccrs.PlateCarree()))

    ax.add_feature(cartopy.feature.LAND, zorder=3, edgecolor='black', facecolor='w')

    cbar = fig.colorbar(im, cax=cax, extend='min', **kw)
    cbar.set_ticks(np.arange(1,13))
    cbar.ax.tick_params(labelsize=14)
    label = cbar.set_label('region number', size=16)

## Global mean vs. 60S-60N time series

In [ ]:
# SST_gm_ctrl      = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_ctrl.nc'     , decode_times=False)
# SST_gm_rcp       = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rcp.nc'      , decode_times=False)
SST_gm_ctrl = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_ctrl.nc' , decode_times=False)
SST_gm_rcp  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_rcp.nc'  , decode_times=False)
SST_gm_lpd  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_lpd.nc'       , decode_times=False)
SST_gm_lpi  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_lpi.nc'       , decode_times=False)

SST_xm_ctrl = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_rect_ctrl.nc', decode_times=False)
SST_xm_rcp  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_rect_rcp.nc' , decode_times=False)
SST_xm_lpd  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_lpd.nc'      , decode_times=False)
SST_xm_lpi  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_lpi.nc'      , decode_times=False)
SST_xm_had  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_had.nc'      , decode_times=False)

In [ ]:
# deseasonalize
SST_gm_ds_ctrl = deseasonalize(SST_gm_ctrl)
SST_gm_ds_rcp  = deseasonalize(SST_gm_rcp )
SST_gm_ds_lpd  = deseasonalize(SST_gm_lpd )
SST_gm_ds_lpi  = deseasonalize(SST_gm_lpi )
SST_xm_ds_ctrl = deseasonalize(SST_xm_ctrl)
SST_xm_ds_rcp  = deseasonalize(SST_xm_rcp )
SST_xm_ds_lpd  = deseasonalize(SST_xm_lpd )
SST_xm_ds_lpi  = deseasonalize(SST_xm_lpi )
SST_xm_ds_had  = deseasonalize(SST_xm_had )

In [ ]:
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot([-1.5,2], [-1  ,2.5], c='k', lw=.5)
plt.plot([-1.5,2], [-1.5,2  ], c='k', lw=.5)
plt.plot([-1.5,2], [-0.5,3  ], c='k', lw=.5)
plt.plot([-1.5,2], [-2  ,1.5], c='k', lw=.5)
plt.xlim((-1,1.5))
plt.ylim((-1,1.5))
plt.scatter(SST_xm_ds_ctrl-SST_xm_ds_ctrl.mean(dim='time'), SST_gm_ds_ctrl-SST_gm_ds_ctrl.mean(dim='time')+.5, alpha=.1)
plt.scatter(SST_xm_ds_rcp -SST_xm_ds_rcp .mean(dim='time'), SST_gm_ds_rcp -SST_gm_ds_rcp .mean(dim='time')   , alpha=.1)
plt.scatter(SST_xm_ds_lpd -SST_xm_ds_lpd .mean(dim='time'), SST_gm_ds_lpd -SST_gm_ds_lpd .mean(dim='time')+1 , alpha=.1)
plt.scatter(SST_xm_ds_lpi -SST_xm_ds_lpi .mean(dim='time'), SST_gm_ds_lpi -SST_gm_ds_lpi .mean(dim='time')-.5, alpha=.1)
plt.scatter(SST_xm_ds_had -SST_xm_ds_had .mean(dim='time'), [-.9]*len(SST_xm_ds_had), alpha=.1)
plt.ylabel('global mean SST' , fontsize=14)
plt.xlabel('60S-60N mean SST', fontsize=14)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot([-1.5,2], [-1  ,2.5], c='k', lw=.5)
plt.plot([-1.5,2], [-1.5,2  ], c='k', lw=.5)
plt.plot([-1.5,2], [-0.5,3  ], c='k', lw=.5)
plt.plot([-1.5,2], [-2  ,1.5], c='k', lw=.5)
plt.xlim((-.3,.3))
plt.ylim((-1.3,1.4))
plt.scatter(SST_xm_ds_ctrl-xr_quadtrend(SST_xm_ds_ctrl), SST_gm_ds_ctrl-xr_quadtrend(SST_gm_ds_ctrl)+.5, alpha=.1)
plt.scatter(SST_xm_ds_rcp -xr_quadtrend(SST_xm_ds_rcp ), SST_gm_ds_rcp -xr_quadtrend(SST_gm_ds_rcp )   , alpha=.1)
plt.scatter(SST_xm_ds_lpd -xr_quadtrend(SST_xm_ds_lpd ), SST_gm_ds_lpd -xr_quadtrend(SST_gm_ds_lpd )+1 , alpha=.1)
plt.scatter(SST_xm_ds_lpi -xr_quadtrend(SST_xm_ds_lpi ), SST_gm_ds_lpi -xr_quadtrend(SST_gm_ds_lpi )-.5, alpha=.1)
plt.scatter(SST_xm_ds_had -xr_quadtrend(SST_xm_ds_had ), [-1]*len(SST_xm_ds_had), alpha=.1)

plt.scatter(SST_xm_ds_lpd[-200*12:] -xr_quadtrend(SST_xm_ds_lpd[-200*12:] ),
            SST_gm_ds_lpd[-200*12:] -xr_quadtrend(SST_gm_ds_lpd[-200*12:] )+1.1 , alpha=.1)
plt.scatter(SST_xm_ds_lpi[-200*12:] -xr_quadtrend(SST_xm_ds_lpi[-200*12:] ),
            SST_gm_ds_lpi[-200*12:] -xr_quadtrend(SST_gm_ds_lpi[-200*12:] )-.6, alpha=.1)
plt.scatter(SST_xm_ds_had[-100*12:] -xr_quadtrend(SST_xm_ds_had[-100*12:] ),
            [-1.1]*(100*12), alpha=.1)
plt.ylabel('global mean SST' , fontsize=14)
plt.xlabel('60S-60N mean SST', fontsize=14)
plt.tight_layout()

In [ ]:
# lowpass deseasonalizing
f, ax = plt.subplots(1, 2, figsize=(12,5))
for i in range(2):
    ax[i].tick_params(labelsize=14)

ax[0].plot(SST_xm_ctrl.time[:120]/12   , SST_xm_ctrl   [:120])
ax[0].plot(SST_xm_ctrl.time[:120]/12   , SST_xm_ds_ctrl[:120])
ax[0].plot(SST_xm_rcp .time[:120]/12+11, SST_xm_rcp    [:120])
ax[0].plot(SST_xm_rcp .time[:120]/12+11, SST_xm_ds_rcp [:120])
ax[0].plot(SST_xm_rcp .time[:120]/12+11, SST_xm_ds_rcp [:120])

ax[1].plot(SST_xm_ctrl.time/12    , SST_xm_ctrl   )
ax[1].plot(SST_xm_ctrl.time/12    , SST_xm_ds_ctrl)
ax[1].plot(SST_xm_rcp .time/12+220, SST_xm_rcp    )
ax[1].plot(SST_xm_rcp .time/12+220, SST_xm_ds_rcp )
ax[1].plot(SST_xm_had .time/365   , SST_xm_had    )
ax[1].plot(SST_xm_had .time/365   , SST_xm_ds_had )
plt.tight_layout()
plt.savefig(f'{path_results}/SST/SST_deseasonalizing_global_mean')

In [ ]:
plt.figure(figsize=(12,5))
plt.tick_params(labelsize=14)

plt.plot(SST_gm_ctrl.time/12 +1950, SST_gm_ctrl   , c='C0', lw=.3, alpha=.5)
plt.plot(SST_gm_rcp .time/12 +2200, SST_gm_rcp    , c='C1', lw=.3, alpha=.5)
plt.plot(SST_gm_lpd .time/365+1350, SST_gm_lpd    , c='C2', lw=.3, alpha=.5)
plt.plot(SST_gm_lpi .time/365-1600, SST_gm_lpi    , c='C3', lw=.3, alpha=.5)
plt.plot(SST_gm_ctrl.time/12 +1950, SST_gm_ds_ctrl, c='C0')
plt.plot(SST_gm_rcp .time/12 +2200, SST_gm_ds_rcp , c='C1')
plt.plot(SST_gm_lpd .time/365+1350, SST_gm_ds_lpd , c='C2')
plt.plot(SST_gm_lpi .time/365-1600, SST_gm_ds_lpi , c='C3')

plt.plot(SST_xm_ctrl.time/12 +1950, SST_xm_ctrl   , c='C0', lw=.3, alpha=.5)
plt.plot(SST_xm_rcp .time/12 +2200, SST_xm_rcp    , c='C1', lw=.3, alpha=.5)
plt.plot(SST_xm_lpd .time/365+1350, SST_xm_lpd    , c='C2', lw=.3, alpha=.5)
plt.plot(SST_xm_lpi .time/365-1600, SST_xm_lpi    , c='C3', lw=.3, alpha=.5)
plt.plot(SST_xm_had .time/365+2350, SST_xm_had    , c='C4', lw=.3, alpha=.5)
plt.plot(SST_xm_ctrl.time/12 +1950, SST_xm_ds_ctrl, c='C0')
plt.plot(SST_xm_rcp .time/12 +2200, SST_xm_ds_rcp , c='C1')
plt.plot(SST_xm_lpd .time/365+1350, SST_xm_ds_lpd , c='C2')
plt.plot(SST_xm_lpi .time/365-1600, SST_xm_ds_lpi , c='C3')
plt.plot(SST_xm_had .time/365+2350, SST_xm_ds_had , c='C4')

plt.ylabel('global / 60S-60N mean SST [$^\circ$C]', fontsize=16)
plt.xlabel('time [years]', fontsize=16)

plt.tight_layout()